In [ ]:
import dtcwt
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
%matplotlib notebook
sns.set_style("white")

from dtcwt.coeffs import biort as _biort, qshift as _qshift
from dtcwt.defaults import DEFAULT_BIORT, DEFAULT_QSHIFT
from dtcwt.utils import appropriate_complex_type_for, asfarray

from dtcwt.numpy.lowlevel import colfilter as colf, coldfilt as cold, colifilt as coli

In [ ]:
im = np.load(os.path.join('tests', 'mandrill.npz'))['mandrill']
plt.imshow(im, cmap='gray', interpolation='none')

In [ ]:
from __future__ import absolute_import, division

__all__ = [ 'colfilter', 'colifilt', 'coldfilt', ]

import numpy as np
from six.moves import xrange
from dtcwt.utils import as_column_vector, asfarray, appropriate_complex_type_for, reflect

def _centered(arr, newsize):
    # Return the center newsize portion of the array.
    # (Shamelessly cribbed from scipy.)
    newsize = np.asanyarray(newsize)
    currsize = np.array(arr.shape)
    startind = (currsize - newsize) // 2
    endind = startind + newsize
    myslice = [slice(startind[k], endind[k]) for k in range(len(endind))]
    return arr[tuple(myslice)]

# This is to allow easy replacement of these later with, possibly, GPU versions
_rfft = np.fft.rfft
_irfft = np.fft.irfft

def _column_convolve(X, h):
    """Convolve the columns of *X* with *h* returning only the 'valid' section,
    i.e. those values unaffected by zero padding. Irrespective of the ftype of
    *h*, the output will have the dtype of *X* appropriately expanded to a
    floating point type if necessary.
    We assume that h is small and so direct convolution is the most efficient.
    """
    Xshape = np.asanyarray(X.shape)
    h = h.flatten().astype(X.dtype)
    h_size = h.shape[0]

    full_size = X.shape[0] + h_size - 1
    Xshape[0] = full_size

    out = np.zeros(Xshape, dtype=X.dtype)
    for idx in xrange(h_size):
        out[idx:(idx+X.shape[0]),...] += X * h[idx]

    outShape = Xshape.copy()
    outShape[0] = abs(X.shape[0] - h_size) + 1
    return _centered(out, outShape)


# Test outputs give expected results

In [ ]:
tf.reset_default_graph()
g = tf.get_default_graph()
dir(g)
g.get_collection('variables')
sess = tf.InteractiveSession(config=tf.ConfigProto(log_device_placement=True))

In [ ]:
#h1o = tf.Variable(f.qshift[0][::-1],trainable=False, dtype=tf.float32)
f = dtcwt.Transform2d()
h1o = tf.constant(f.qshift[0][::-1],dtype=tf.float32)
    
in_ = tf.placeholder(tf.float32, shape=[None, 512, 512])
init_op = tf.global_variables_initializer()

qshift = f.qshift[0].astype('float32')
im_hat = colf(colf(colf(im, qshift),qshift),qshift)
y1 = colfilter(colfilter(colfilter(in_, h1o), h1o), h1o)
y2 = rowfilter(rowfilter(rowfilter(in_, h1o), h1o), h1o)

In [ ]:
# Compare the 2
sess.run(init_op)
im_hat1 = sess.run(y1, feed_dict={in_:[im]})[0]
im_hat2 = sess.run(y2, feed_dict={in_:[im.T]})[0]
    
np.testing.assert_array_almost_equal(im_hat, im_hat1, decimal=4)
np.testing.assert_array_almost_equal(im_hat, im_hat2.T, decimal=4)

# Compare the execution times for direct filtering and GPU filtering

In [ ]:
import time
h = f.qshift[0].astype('float32')
time1 = time.time()
for i in range(1000):
    colf(colf(colf(im, h),h),h)
time2 = time.time()
print('Took {:3f} ms'.format((time2-time1)*1000.0))

In [ ]:
batch = np.stack([im]*100,axis=0)

time1 = time.time()
for i in range(10):
    b = sess.run(y1, feed_dict={in_:batch})
time2 = time.time()
print('Took {:3f} ms'.format((time2-time1)*1000.0))

In [ ]:
batch = np.stack([im]*100,axis=0)

time1 = time.time()
for i in range(10):
    b = sess.run(y2, feed_dict={in_:batch})
time2 = time.time()
print('Took {:3f} ms'.format((time2-time1)*1000.0))

# Redefine the Transform

In [ ]:
class Transform2d(object):
    """
    An implementation of the 2D DT-CWT via Tensorflow. 
    *biort* and *qshift* are the wavelets which parameterise the transform.
    If *biort* or *qshift* are strings, they are used as an argument to the
    :py:func:`dtcwt.coeffs.biort` or :py:func:`dtcwt.coeffs.qshift` functions.
    Otherwise, they are interpreted as tuples of vectors giving filter
    coefficients. In the *biort* case, this should be (h0o, g0o, h1o, g1o). In
    the *qshift* case, this should be (h0a, h0b, g0a, g0b, h1a, h1b, g1a, g1b).
    
    Creating an object of this class loads the necessary filters onto the 
    tensorflow graph. A subsequent call to :py:func:`Transform2d.forward` with 
    a placeholder will create a forward transform for an input of the placeholder's
    size. You can evaluate the resulting ops several times feeding different
    images into the placeholder *assuming* they have the same resolution. For 
    a different resolution image, call the :py:func:`Transform2d.forward` 
    function again.
    """

    def __init__(self, biort=DEFAULT_BIORT, qshift=DEFAULT_QSHIFT):
        # Load bi-orthogonal wavelets
        try:
            self.biort = _biort(biort)
        except TypeError:
            self.biort = biort

        # Load quarter sample shift wavelets
        try:
            self.qshift = _qshift(qshift)
        except TypeError:
            self.qshift = qshift

        ### Load the ops onto the graph for the filter banks

        # If biort has 6 elements instead of 4, then it's a modified
        # rotationally symmetric wavelet
        # h0o - analysis low pass filter
        # g0o - synthesis low pass filter
        # h1o - analysis high pass filter
        # g1o - synthesis high pass filter
        # h2o - analysis band pass filter for 45 deg wavelets
        # g2o - synthesis band pass filter for 45 deg wavelets
        if len(self.biort) == 4:
            # h0o, g0o, h1o, g1o = self.biort            
            self.h0o = tf.constant(self.biort[0], dtype=tf.float32, name='dtcwt/h0o')
            self.g0o = tf.constant(self.biort[1], dtype=tf.float32, name='dtcwt/g0o')
            self.h1o = tf.constant(self.biort[2], dtype=tf.float32, name='dtcwt/h1o')
            self.g1o = tf.constant(self.biort[3], dtype=tf.float32, name='dtcwt/g1o')
        elif len(self.biort) == 6:
            #h0o, g0o, h1o, g1o, h2o, g2o = self.biort
            self.h0o = tf.constant(self.biort[0], dtype=tf.float32, name='dtcwt/h0o')
            self.g0o = tf.constant(self.biort[1], dtype=tf.float32, name='dtcwt/g0o')
            self.h1o = tf.constant(self.biort[2], dtype=tf.float32, name='dtcwt/h1o')
            self.g1o = tf.constant(self.biort[3], dtype=tf.float32, name='dtcwt/g1o')
            self.h2o = tf.constant(self.biort[4], dtype=tf.float32, name='dtcwt/h2o')
            self.g2o = tf.constant(self.biort[5], dtype=tf.float32, name='dtcwt/g2o')
        else:
            raise ValueError('Biort wavelet must have 6 or 4 components.')

            
        # If qshift has 12 elements instead of 8, then it's a modified
        # rotationally symmetric wavelet        
        # h0a - analysis low pass filter tree a
        # h0b - analysis low pass filter tree b
        # h1a - analysis high pass filter tree a
        # h1b - analysis high pass filter tree b
        # h2a - analysis band pass filter tree a (for 45 deg wavelets)
        # h2b - analysis band pass filter tree b (for 45 deg wavelets)
        # g.. - synthesis equivalents
        
        # We have to reverse the qshift filters, as tensorflow's conv2d is
        # really cross-correlation. Note that we didn't have to do this for
        # biorthogonal filters as they are already symmetric.
        if len(self.qshift) == 8:
            #h0a, h0b, g0a, g0b, h1a, h1b, g1a, g1b = self.qshift
            self.h0a = tf.constant(self.qshift[0][::-1], dtype=tf.float32, name='dtcwt/h0a')
            self.h0b = tf.constant(self.qshift[1][::-1], dtype=tf.float32, name='dtcwt/h0b')
            self.g0a = tf.constant(self.qshift[2][::-1], dtype=tf.float32, name='dtcwt/g0a')
            self.g0b = tf.constant(self.qshift[3][::-1], dtype=tf.float32, name='dtcwt/g0b')
            self.h1a = tf.constant(self.qshift[4][::-1], dtype=tf.float32, name='dtcwt/h1a')
            self.h1b = tf.constant(self.qshift[5][::-1], dtype=tf.float32, name='dtcwt/h1b')
            self.g1a = tf.constant(self.qshift[6][::-1], dtype=tf.float32, name='dtcwt/g1a')
            self.g1b = tf.constant(self.qshift[7][::-1], dtype=tf.float32, name='dtcwt/g1b')
        elif len(self.qshift) == 12:
            #h0a, h0b, g0a, g0b, h1a, h1b, g1a, g1b, h2a, h2b = self.qshift[:10]
            self.h0a = tf.constant(self.qshift[0][::-1], dtype=tf.float32, name='dtcwt/h0a')
            self.h0b = tf.constant(self.qshift[1][::-1], dtype=tf.float32, name='dtcwt/h0b')
            self.g0a = tf.constant(self.qshift[2][::-1], dtype=tf.float32, name='dtcwt/g0a')
            self.g0b = tf.constant(self.qshift[3][::-1], dtype=tf.float32, name='dtcwt/g0b')
            self.h1a = tf.constant(self.qshift[4][::-1], dtype=tf.float32, name='dtcwt/h1a')
            self.h1b = tf.constant(self.qshift[5][::-1], dtype=tf.float32, name='dtcwt/h1b')
            self.g1a = tf.constant(self.qshift[6][::-1], dtype=tf.float32, name='dtcwt/g1a')
            self.g1b = tf.constant(self.qshift[7][::-1], dtype=tf.float32, name='dtcwt/g1b')
            self.h2a = tf.constant(self.qshift[8][::-1], dtype=tf.float32, name='dtcwt/h2a')
            self.h2b = tf.constant(self.qshift[9][::-1], dtype=tf.float32, name='dtcwt/h2b')
        else:
            raise ValueError('Qshift wavelet must have 12 or 8 components.')

            
            
    def forward(self, X, nlevels=3, include_scale=False):
        """Perform a *n*-level DTCWT-2D decompostion on a 2D matrix *X*.
        :param X: 3D real array of size [Batch, rows, cols]
        :param nlevels: Number of levels of wavelet decomposition
        :param include_scale: True if you want to receive the lowpass coefficients at
            intermediate layers.
        :returns: A :py:class:`dtcwt.Pyramid` compatible object representing the transform-domain signal
        .. codeauthor:: Fergal Cotter <fbc23@cam.ac.uk>, Feb 2017
        .. codeauthor:: Rich Wareham <rjw57@cantab.net>, Aug 2013
        .. codeauthor:: Nick Kingsbury, Cambridge University, Sept 2001
        .. codeauthor:: Cian Shaffrey, Cambridge University, Sept 2001
        """

        # Check the shape of the input
        original_size = X.get_shape().as_list()[1:]
        
        if len(original_size) >= 3:
            raise ValueError('The entered image is {0}, please enter each image slice separately.'.
                             format('x'.join(list(str(s) for s in X.shape))))


        ############################## Resize #################################
        # The next few lines of code check to see if the image is odd in size, 
        # if so an extra ... row/column will be added to the bottom/right of the 
        # image
        initial_row_extend = 0  #initialise
        initial_col_extend = 0
        if original_size[0] % 2 != 0:
            # if X.shape[0] is not divisable by 2 then we need to extend X by 
            # adding a row at the bottom
            bottom_row = tf.slice(X, [0, original_size[0] - 1, 0], [-1, 1, -1])
            X = tf.concat([X, bottom_row], axis=1)
            initial_row_extend = 1

        if original_size[1] % 2 != 0:
            # if X.shape[1] is not divisable by 2 then we need to extend X by 
            # adding a col to the right
            right_col = tf.slice(X, [0, 0, original_size[1] - 1], [-1, -1, 1])
            X = tf.concat([X, right_col], axis=2)
            initial_col_extend = 1

        extended_size = X.get_shape().as_list()[1:3]

        if nlevels == 0:
            if include_scale:
                return Pyramid_ops(X, (), ())
            else:
                return Pyramid_ops(X, ())

        
        ############################ Initialise ###############################
        Yh = [None,] * nlevels
        if include_scale:
            # this is only required if the user specifies a third output component.
            Yscale = [None,] * nlevels

        ############################# Level 1 #################################
        # Uses the biorthogonal filters
        if nlevels >= 1:
            # Do odd top-level filters on cols.
            Lo = colfilter(X, self.h0o)
            Hi = colfilter(X, self.h1o)
            if len(self.biort) >= 6:
                Ba = colfilter(X, self.h2o)

            # Do odd top-level filters on rows.
            LoLo = rowfilter(Lo, self.h0o)
            LoLo_shape = LoLo.get_shape().as_list()[1:3]            
            
            # Horizontal wavelet pair (15 & 165 degrees)
            horiz = q2c(rowfilter(Hi, self.h0o))  
            
            # Vertical wavelet pair (75 & 105 degrees)
            vertic = q2c(rowfilter(Lo, self.h1o))  
            
            # Diagonal wavelet pair (45 & 135 degrees)
            if len(self.biort) >= 6:
                diag = q2c(rowfilter(Ba, self.h2o))  
            else:
                diag = q2c(rowfilter(Hi, self.h1o))  
            
            # Pack all 6 tensors into one 
            Yh[0] = tf.stack(
                [horiz[0], diag[0], vertic[0], vertic[1], diag[1], horiz[1]],
                axis=3)
            
            if include_scale:
                Yscale[0] = LoLo
                
                
        ############################# Level 2+ ################################
        # Uses the qshift filters        
        for level in xrange(1, nlevels):
            row_size, col_size = LoLo_shape[0], LoLo_shape[1]
            if row_size % 4 != 0:
                bottom_row = tf.slice(LoLo, [0, row_size - 2, 0], [-1, 2, -1])
                LoLo = tf.concat([LoLo, bottom_row], axis=1)

            if col_size % 4 != 0:
                right_col = tf.slice(LoLo, [0, 0, col_size - 2], [-1, -1, 2])
                LoLo = tf.concat([LoLo, right_col], axis=2)

            # Do even Qshift filters on cols.
            Lo = coldfilt(LoLo, self.h0b, self.h0a)
            Hi = coldfilt(LoLo, self.h1b, self.h1a)
            if len(self.qshift) >= 12:
                Ba = coldfilt(LoLo, self.h2b, self.h2a)

            # Do even Qshift filters on rows.
            LoLo = rowdfilt(Lo, self.h0b, self.h0a)
            LoLo_shape = LoLo.get_shape().as_list()[1:3]            
            
            # Horizontal wavelet pair (15 & 165 degrees)
            horiz = q2c(rowdfilt(Hi, self.h0b, self.h0a))  
            
            # Vertical wavelet pair (75 & 105 degrees)
            vertic = q2c(rowdfilt(Lo, self.h1b, self.h1a))  
            
            # Diagonal wavelet pair (45 & 135 degrees)
            if len(self.qshift) >= 12:
                diag = q2c(rowdfilt(Ba, self.h2b, self.h2a))  
            else:
                diag = q2c(rowdfilt(Hi, self.h1b, self.h1a))  
            
            # Pack all 6 tensors into one 
            Yh[level] = tf.stack(
                [horiz[0], diag[0], vertic[0], vertic[1], diag[1], horiz[1]],
                axis=3)
            
            if include_scale:
                Yscale[level] = LoLo
            
        Yl = LoLo
        
        if initial_row_extend == 1 and initial_col_extend == 1:
            logging.warn('The image entered is now a {0} NOT a {1}.'.format(
                'x'.join(list(str(s) for s in extended_size)),
                'x'.join(list(str(s) for s in original_size))))
            logging.warn(
                'The bottom row and rightmost column have been duplicated, prior to decomposition.')

        if initial_row_extend == 1 and initial_col_extend == 0:
            logging.warn('The image entered is now a {0} NOT a {1}.'.format(
                'x'.join(list(str(s) for s in extended_size)),
                'x'.join(list(str(s) for s in original_size))))
            logging.warn(
                'The bottom row has been duplicated, prior to decomposition.')

        if initial_row_extend == 0 and initial_col_extend == 1:
            logging.warn('The image entered is now a {0} NOT a {1}.'.format(
                'x'.join(list(str(s) for s in extended_size)),
                'x'.join(list(str(s) for s in original_size))))
            logging.warn(
                'The rightmost column has been duplicated, prior to decomposition.')

        if include_scale:
            return Pyramid_ops(Yl, tuple(Yh), tuple(Yscale))
        else:
            return Pyramid_ops(Yl, tuple(Yh))
        

def q2c(y):
    """
    Convert from quads in y to complex numbers in z.
    """

    # Arrange pixels from the corners of the quads into
    # 2 subimages of alternate real and imag pixels.
    #  a----b
    #  |    |
    #  |    |
    #  c----d
    # Combine (a,b) and (d,c) to form two complex subimages.
    a,b,c,d = y[:, 0::2, 0::2], y[:, 0::2,1::2], y[:, 1::2,0::2], y[:, 1::2,1::2]
    
    p = tf.complex(a/np.sqrt(2), b/np.sqrt(2))    # p = (a + jb) / sqrt(2)
    q = tf.complex(d/np.sqrt(2), -c/np.sqrt(2))   # q = (d - jc) / sqrt(2)

    # Form the 2 highpasses in z.
    return (p-q, p+q)        

In [ ]:
def med_level(X, h0b, h0a, h1b, h1a):
    # Do even Qshift filters on cols.
    Lo = coldfilt(X, h0b, h0a)
    Hi = coldfilt(X, h1b, h1a)
    if False >= 12:
        Ba = coldfilt(X, h2b, h2a)

    # Do even Qshift filters on rows.
    LoLo = rowdfilt(Lo, h0b, h0a)
    LoLo_shape = LoLo.get_shape().as_list()[1:3]            

    # Horizontal wavelet pair (15 & 165 degrees)
    horiz = q2c(rowdfilt(Hi, h0b, h0a))  

    # Vertical wavelet pair (75 & 105 degrees)
    vertic = q2c(rowdfilt(Lo, h1b, h1a))  

    # Diagonal wavelet pair (45 & 135 degrees)
    if False >= 12:
        diag = q2c(rowdfilt(Ba, h2b, h2a))  
    else:
        diag = q2c(rowdfilt(Hi, h1b, h1a))  

    # Pack all 6 tensors into one 
    Yh = tf.stack(
        [horiz[0], diag[0], vertic[0], vertic[1], diag[1], horiz[1]],
        axis=3)
    
    return LoLo, Yh

In [ ]:
a = tf.reduce_sum(f.h1b*f.h1a)
print(sess.run(a))

In [ ]:
in_t = tf.expand_dims(tf.constant(p2.scales[0]),axis=0)
a_op, b_op = med_level(in_t, f.h0b, f.h0a, f.h1b, f.h1a)
a, b = sess.run([a_op,b_op])
np.testing.assert_array_almost_equal(a[0], p2.scales[1], decimal=4)
np.testing.assert_array_almost_equal(b[0], p2.highpasses[1], decimal=4)

In [ ]:
tf.reset_default_graph()
sess = tf.InteractiveSession(config=tf.ConfigProto(log_device_placement=True))

f = Transform2d(biort='near_sym_a',qshift='qshift_b')    
f2 = dtcwt.Transform2d(biort='near_sym_a',qshift='qshift_b')
in_ = tf.placeholder(tf.float32, shape=[None, 512, 512])


h0b, h0a = sess.run([f.h0b, f.h0a])
out_1 = cold(im.T,h0b[::-1],h0a[::-1]).T
out_2 = cold(im, h0b[::-1],h0a[::-1])
in_t = tf.expand_dims(tf.constant(im),axis=0)
a_r = rowdfilt(in_t,f.h0b,f.h0a)
a_c = coldfilt(in_t,f.h0b,f.h0a)
a,a2 = sess.run([a_r,a_c])
np.testing.assert_array_almost_equal(a[0], out_1, decimal=4)
np.testing.assert_array_almost_equal(a2[0], out_2, decimal=4)

In [ ]:
a.shape

In [ ]:
p_op = f.forward(in_, nlevels=3)
p2 = f2.forward(im, nlevels=3,include_scale=True)
p = p_op.eval(sess, in_, [im])

lo = p.lowpass[0]
hi1 = p.highpasses[1][0]
hi2 = p2.highpasses[1]

In [ ]:
# Check that the results are the same
print(p.lowpass.shape)
print(p2.lowpass.shape)
print(p.highpasses[0].shape)
print(p2.highpasses[0].shape)
np.testing.assert_array_almost_equal(p.lowpass[0], p2.lowpass, decimal=4)
for i in range(3):
    np.testing.assert_array_almost_equal(p.highpasses[i][0], p2.highpasses[i], decimal=4)

In [ ]:
b = tf.reshape(f.h0a, [1,5,2,1])
c = tf.stack([f.h0a[0::2], f.h0a[1::2]], axis=-1)
c = tf.reshape(c, [5,2])
a_e, b_e, c_e = sess.run([f.h0a, b,c])

In [ ]:
# Interleaving Columns
a = np.random.randn(10,2)
b = np.random.randn(10,2)
Y = np.zeros((20,2))
Y[0::2,:], Y[1::2,:] = a,b
a_t = tf.constant(a,dtype=tf.float32)
b_t = tf.constant(b,dtype=tf.float32)
Y_t1 = tf.stack([a_t,b_t], axis=1)
Y_t = tf.reshape(Y_t1, [20,2])
Y2 = sess.run(Y_t)
np.testing.assert_array_almost_equal(Y, Y2, decimal=4)

In [ ]:
Y_t2 = tf.reshape(Y_t, [10,2,2])
a2_t, b2_t = tf.unstack(Y_t2,axis=1)
a3_t, b3_t = Y_t[0::2,:], Y_t[1::2,:]
a2,b2 = sess.run([a2_t,b2_t])
a3,b3 = sess.run([a3_t,b3_t])
t = np.arange(0,10,2,dtype=np.int32)
a4_t = Y_t[t,:]
a4 = sess.run(a4_t)

In [ ]:
"""
        if len(X.shape) >= 3:
            raise ValueError('The entered image is {0}, please enter each image slice separately.'.
                    format('x'.join(list(str(s) for s in X.shape))))

        
        for level in xrange(1, nlevels):
            row_size, col_size = LoLo.shape
            if row_size % 4 != 0:
                # Extend by 2 rows if no. of rows of LoLo are not divisable by 4
                LoLo = np.vstack((LoLo[:1,:], LoLo, LoLo[-1:,:]))

            if col_size % 4 != 0:
                # Extend by 2 cols if no. of cols of LoLo are not divisable by 4
                LoLo = np.hstack((LoLo[:,:1], LoLo, LoLo[:,-1:]))

            # Do even Qshift filters on rows.
            Lo = coldfilt(LoLo,h0b,h0a).T
            Hi = coldfilt(LoLo,h1b,h1a).T
            if len(self.qshift) >= 12:
                Ba = coldfilt(LoLo,h2b,h2a).T

            # Do even Qshift filters on columns.
            LoLo = coldfilt(Lo,h0b,h0a).T

            Yh[level] = np.zeros((LoLo.shape[0]>>1, LoLo.shape[1]>>1, 6), dtype=complex_dtype)
            Yh[level][:,:,0:6:5] = q2c(coldfilt(Hi,h0b,h0a).T)  # Horizontal
            Yh[level][:,:,2:4:1] = q2c(coldfilt(Lo,h1b,h1a).T)  # Vertical
            if len(self.qshift) >= 12:
                Yh[level][:,:,1:5:3] = q2c(coldfilt(Ba,h2b,h2a).T)  # Diagonal   
            else:
                Yh[level][:,:,1:5:3] = q2c(coldfilt(Hi,h1b,h1a).T)  # Diagonal   

            if include_scale:
                Yscale[level] = LoLo

        Yl = LoLo

        if initial_row_extend == 1 and initial_col_extend == 1:
            logging.warn('The image entered is now a {0} NOT a {1}.'.format(
                'x'.join(list(str(s) for s in extended_size)),
                'x'.join(list(str(s) for s in original_size))))
            logging.warn(
                'The bottom row and rightmost column have been duplicated, prior to decomposition.')

        if initial_row_extend == 1 and initial_col_extend == 0:
            logging.warn('The image entered is now a {0} NOT a {1}.'.format(
                'x'.join(list(str(s) for s in extended_size)),
                'x'.join(list(str(s) for s in original_size))))
            logging.warn(
                'The bottom row has been duplicated, prior to decomposition.')

        if initial_row_extend == 0 and initial_col_extend == 1:
            logging.warn('The image entered is now a {0} NOT a {1}.'.format(
                'x'.join(list(str(s) for s in extended_size)),
                'x'.join(list(str(s) for s in original_size))))
            logging.warn(
                'The rightmost column has been duplicated, prior to decomposition.')

        if include_scale:
            return Pyramid(Yl, tuple(Yh), tuple(Yscale))
        else:
            return Pyramid(Yl, tuple(Yh))
"""

#    def inverse(self, pyramid, gain_mask=None):
        """Perform an *n*-level dual-tree complex wavelet (DTCWT) 2D
        reconstruction.
        :param pyramid: A :py:class:`dtcwt.Pyramid`-like class holding the transform domain representation to invert.
        :param gain_mask: Gain to be applied to each subband.
        :returns: A numpy-array compatible instance with the reconstruction.
        The (*d*, *l*)-th element of *gain_mask* is gain for subband with direction
        *d* at level *l*. If gain_mask[d,l] == 0, no computation is performed for
        band (d,l). Default *gain_mask* is all ones. Note that both *d* and *l* are
        zero-indexed.
        .. codeauthor:: Rich Wareham <rjw57@cantab.net>, Aug 2013
        .. codeauthor:: Nick Kingsbury, Cambridge University, May 2002
        .. codeauthor:: Cian Shaffrey, Cambridge University, May 2002
        """
"""
        Yl = pyramid.lowpass
        Yh = pyramid.highpasses

        a = len(Yh) # No of levels.

        if gain_mask is None:
            gain_mask = np.ones((6,a)) # Default gain_mask.

        gain_mask = np.array(gain_mask)

        # If biort has 6 elements instead of 4, then it's a modified
        # rotationally symmetric wavelet
        # FIXME: there's probably a nicer way to do this
        if len(self.biort) == 4:
            h0o, g0o, h1o, g1o = self.biort
        elif len(self.biort) == 6:
            h0o, g0o, h1o, g1o, h2o, g2o = self.biort
        else:
            raise ValueError('Biort wavelet must have 6 or 4 components.')

        # If qshift has 12 elements instead of 8, then it's a modified
        # rotationally symmetric wavelet
        # FIXME: there's probably a nicer way to do this
        if len(self.qshift) == 8:
            h0a, h0b, g0a, g0b, h1a, h1b, g1a, g1b = self.qshift
        elif len(self.qshift) == 12:
            h0a, h0b, g0a, g0b, h1a, h1b, g1a, g1b, h2a, h2b, g2a, g2b = self.qshift
        else:
            raise ValueError('Qshift wavelet must have 12 or 8 components.')

        current_level = a
        Z = Yl

        while current_level >= 2: # this ensures that for level 1 we never do the following
            lh = c2q(Yh[current_level-1][:,:,[0, 5]], gain_mask[[0, 5], current_level-1])
            hl = c2q(Yh[current_level-1][:,:,[2, 3]], gain_mask[[2, 3], current_level-1])
            hh = c2q(Yh[current_level-1][:,:,[1, 4]], gain_mask[[1, 4], current_level-1])

            # Do even Qshift filters on columns.
            y1 = colifilt(Z,g0b,g0a) + colifilt(lh,g1b,g1a)

            if len(self.qshift) >= 12:
                y2 = colifilt(hl,g0b,g0a)
                y2bp = colifilt(hh,g2b,g2a)

                # Do even Qshift filters on rows.
                Z = (colifilt(y1.T,g0b,g0a) + colifilt(y2.T,g1b,g1a) + colifilt(y2bp.T, g2b, g2a)).T
            else:
                y2 = colifilt(hl,g0b,g0a) + colifilt(hh,g1b,g1a)

                # Do even Qshift filters on rows.
                Z = (colifilt(y1.T,g0b,g0a) + colifilt(y2.T,g1b,g1a)).T

            # Check size of Z and crop as required
            [row_size, col_size] = Z.shape
            S = 2*np.array(Yh[current_level-2].shape)
            if row_size != S[0]:    # check to see if this result needs to be cropped for the rows
                Z = Z[1:-1,:]
            if col_size != S[1]:    # check to see if this result needs to be cropped for the cols
                Z = Z[:,1:-1]

            if np.any(np.array(Z.shape) != S[:2]):
                raise ValueError('Sizes of highpasses are not valid for DTWAVEIFM2')
            
            current_level = current_level - 1

        if current_level == 1:
            lh = c2q(Yh[current_level-1][:,:,[0, 5]],gain_mask[[0, 5],current_level-1])
            hl = c2q(Yh[current_level-1][:,:,[2, 3]],gain_mask[[2, 3],current_level-1])
            hh = c2q(Yh[current_level-1][:,:,[1, 4]],gain_mask[[1, 4],current_level-1])

            # Do odd top-level filters on columns.
            y1 = colfilter(Z,g0o) + colfilter(lh,g1o)

            if len(self.biort) >= 6:
                y2 = colfilter(hl,g0o)
                y2bp = colfilter(hh,g2o)

                # Do odd top-level filters on rows.
                Z = (colfilter(y1.T,g0o) + colfilter(y2.T,g1o) + colfilter(y2bp.T, g2o)).T
            else:
                y2 = colfilter(hl,g0o) + colfilter(hh,g1o)

                # Do odd top-level filters on rows.
                Z = (colfilter(y1.T,g0o) + colfilter(y2.T,g1o)).T

        return Z
"""
#==========================================================================================
#                       **********    INTERNAL FUNCTIONS    **********
#==========================================================================================



def c2q(w,gain):
    """
    Scale by gain and convert from complex w(:,:,1:2) to real quad-numbers
    in z.
    Arrange pixels from the real and imag parts of the 2 highpasses
    into 4 separate subimages .
     A----B     Re   Im of w(:,:,1)
     |    |
     |    |
     C----D     Re   Im of w(:,:,2)
    """

    x = np.zeros((w.shape[0] << 1, w.shape[1] << 1), dtype=w.real.dtype)

    sc = np.sqrt(0.5) * gain
    P = w[:,:,0]*sc[0] + w[:,:,1]*sc[1]
    Q = w[:,:,0]*sc[0] - w[:,:,1]*sc[1]

    # Recover each of the 4 corners of the quads.
    x[0::2, 0::2] = P.real  # a = (A+C)*sc
    x[0::2, 1::2] = P.imag  # b = (B+D)*sc
    x[1::2, 0::2] = Q.imag  # c = (B-D)*sc
    x[1::2, 1::2] = -Q.real # d = (C-A)*sc

    return x

In [ ]:
def _centered(arr, newsize):
    # Return the center newsize portion of the array.
    # (Shamelessly cribbed from scipy.)
    newsize = np.asanyarray(newsize)
    currsize = np.array(arr.shape)
    startind = (currsize - newsize) // 2
    endind = startind + newsize
    myslice = [slice(startind[k], endind[k]) for k in range(len(endind))]
    return arr[tuple(myslice)]

# This is to allow easy replacement of these later with, possibly, GPU versions
_rfft = np.fft.rfft
_irfft = np.fft.irfft

def _column_convolve(X, h):
    """Convolve the columns of *X* with *h* returning only the 'valid' section,
    i.e. those values unaffected by zero padding. Irrespective of the ftype of
    *h*, the output will have the dtype of *X* appropriately expanded to a
    floating point type if necessary.
    We assume that h is small and so direct convolution is the most efficient.
    """
    Xshape = np.asanyarray(X.shape)
    h = h.flatten().astype(X.dtype)
    h_size = h.shape[0]

    full_size = X.shape[0] + h_size - 1
    Xshape[0] = full_size

    out = np.zeros(Xshape, dtype=X.dtype)
    for idx in xrange(h_size):
        out[idx:(idx+X.shape[0]),...] += X * h[idx]

    outShape = Xshape.copy()
    outShape[0] = abs(X.shape[0] - h_size) + 1
    return _centered(out, outShape)


    return Y

def coldfilt(X, ha, hb):
    """Filter the columns of image X using the two filters ha and hb =
    reverse(ha).  ha operates on the odd samples of X and hb on the even
    samples.  Both filters should be even length, and h should be approx linear
    phase with a quarter sample advance from its mid pt (i.e. :math:`|h(m/2)| >
    |h(m/2 + 1)|`).
    .. code-block:: text
                          ext        top edge                     bottom edge       ext
        Level 1:        !               |               !               |               !
        odd filt on .    b   b   b   b   a   a   a   a   a   a   a   a   b   b   b   b
        odd filt on .      a   a   a   a   b   b   b   b   b   b   b   b   a   a   a   a
        Level 2:        !               |               !               |               !
        +q filt on x      b       b       a       a       a       a       b       b
        -q filt on o          a       a       b       b       b       b       a       a
    The output is decimated by two from the input sample rate and the results
    from the two filters, Ya and Yb, are interleaved to give Y.  Symmetric
    extension with repeated end samples is used on the composite X columns
    before each filter is applied.
    Raises ValueError if the number of rows in X is not a multiple of 4, the
    length of ha does not match hb or the lengths of ha or hb are non-even.
    .. codeauthor:: Rich Wareham <rjw57@cantab.net>, August 2013
    .. codeauthor:: Cian Shaffrey, Cambridge University, August 2000
    .. codeauthor:: Nick Kingsbury, Cambridge University, August 2000
    """
    # Make sure all inputs are arrays
    X = asfarray(X)
    ha = asfarray(ha)
    hb = asfarray(hb)

    r, c = X.shape
    if r % 4 != 0:
        raise ValueError('No. of rows in X must be a multiple of 4')

    if ha.shape != hb.shape:
        raise ValueError('Shapes of ha and hb must be the same')

    if ha.shape[0] % 2 != 0:
        raise ValueError('Lengths of ha and hb must be even')

    m = ha.shape[0]
    m2 = np.fix(m*0.5)

    # Set up vector for symmetric extension of X with repeated end samples.
    xe = reflect(np.arange(-m, r+m), -0.5, r-0.5)

    # Select odd and even samples from ha and hb. Note that due to 0-indexing
    # 'odd' and 'even' are not perhaps what you might expect them to be.
    hao = as_column_vector(ha[0:m:2])
    hae = as_column_vector(ha[1:m:2])
    hbo = as_column_vector(hb[0:m:2])
    hbe = as_column_vector(hb[1:m:2])
    t = np.arange(5, r+2*m-2, 4)
    r2 = r//2;
    Y = np.zeros((r2,c), dtype=X.dtype)

    if np.sum(ha*hb) > 0:
       s1 = slice(0, r2, 2)
       s2 = slice(1, r2, 2)
    else:
       s2 = slice(0, r2, 2)
       s1 = slice(1, r2, 2)

    # Perform filtering on columns of extended matrix X(xe,:) in 4 ways.
    Y[s1,:] = _column_convolve(X[xe[t-1],:],hao) + _column_convolve(X[xe[t-3],:],hae)
    Y[s2,:] = _column_convolve(X[xe[t],:],hbo) + _column_convolve(X[xe[t-2],:],hbe)

    return Y

def colifilt(X, ha, hb):
    """ Filter the columns of image X using the two filters ha and hb =
    reverse(ha).  ha operates on the odd samples of X and hb on the even
    samples.  Both filters should be even length, and h should be approx linear
    phase with a quarter sample advance from its mid pt (i.e `:math:`|h(m/2)| >
    |h(m/2 + 1)|`).
    .. code-block:: text
                          ext       left edge                      right edge       ext
        Level 2:        !               |               !               |               !
        +q filt on x      b       b       a       a       a       a       b       b
        -q filt on o          a       a       b       b       b       b       a       a
        Level 1:        !               |               !               |               !
        odd filt on .    b   b   b   b   a   a   a   a   a   a   a   a   b   b   b   b
        odd filt on .      a   a   a   a   b   b   b   b   b   b   b   b   a   a   a   a
    The output is interpolated by two from the input sample rate and the
    results from the two filters, Ya and Yb, are interleaved to give Y.
    Symmetric extension with repeated end samples is used on the composite X
    columns before each filter is applied.
    .. codeauthor:: Rich Wareham <rjw57@cantab.net>, August 2013
    .. codeauthor:: Cian Shaffrey, Cambridge University, August 2000
    .. codeauthor:: Nick Kingsbury, Cambridge University, August 2000
    """
    # Make sure all inputs are arrays
    X = asfarray(X)
    ha = asfarray(ha)
    hb = asfarray(hb)

    r, c = X.shape
    if r % 2 != 0:
        raise ValueError('No. of rows in X must be a multiple of 2')

    if ha.shape != hb.shape:
        raise ValueError('Shapes of ha and hb must be the same')

    if ha.shape[0] % 2 != 0:
        raise ValueError('Lengths of ha and hb must be even')

    m = ha.shape[0]
    m2 = np.fix(m*0.5)

    Y = np.zeros((r*2,c), dtype=X.dtype)
    if not np.any(np.nonzero(X[:])[0]):
        return Y

    if m2 % 2 == 0:
        # m/2 is even, so set up t to start on d samples.
        # Set up vector for symmetric extension of X with repeated end samples.
        # Use 'reflect' so r < m2 works OK.
        xe = reflect(np.arange(-m2, r+m2, dtype=np.int), -0.5, r-0.5)

        t = np.arange(3, r+m, 2)
        if np.sum(ha*hb) > 0:
            ta = t
            tb = t - 1
        else:
            ta = t - 1
            tb = t

        # Select odd and even samples from ha and hb. Note that due to 0-indexing
        # 'odd' and 'even' are not perhaps what you might expect them to be.
        hao = as_column_vector(ha[0:m:2])
        hae = as_column_vector(ha[1:m:2])
        hbo = as_column_vector(hb[0:m:2])
        hbe = as_column_vector(hb[1:m:2])

        s = np.arange(0,r*2,4)

        Y[s,:]   = _column_convolve(X[xe[tb-2],:],hae)
        Y[s+1,:] = _column_convolve(X[xe[ta-2],:],hbe)
        Y[s+2,:] = _column_convolve(X[xe[tb  ],:],hao)
        Y[s+3,:] = _column_convolve(X[xe[ta  ],:],hbo)
    else:
        # m/2 is odd, so set up t to start on b samples.
        # Set up vector for symmetric extension of X with repeated end samples.
        # Use 'reflect' so r < m2 works OK.
        xe = reflect(np.arange(-m2, r+m2, dtype=np.int), -0.5, r-0.5)

        t = np.arange(2, r+m-1, 2)
        if np.sum(ha*hb) > 0:
            ta = t
            tb = t - 1
        else:
            ta = t - 1
            tb = t

        # Select odd and even samples from ha and hb. Note that due to 0-indexing
        # 'odd' and 'even' are not perhaps what you might expect them to be.
        hao = as_column_vector(ha[0:m:2])
        hae = as_column_vector(ha[1:m:2])
        hbo = as_column_vector(hb[0:m:2])
        hbe = as_column_vector(hb[1:m:2])

        s = np.arange(0,r*2,4)

        Y[s,:]   = _column_convolve(X[xe[tb],:],hao)
        Y[s+1,:] = _column_convolve(X[xe[ta],:],hbo)
        Y[s+2,:] = _column_convolve(X[xe[tb],:],hae)
        Y[s+3,:] = _column_convolve(X[xe[ta],:],hbe)

    return Y

In [ ]:
def colfilter(X, h):
    """Filter the columns of image *X* using filter vector *h*, without decimation.
    If len(h) is odd, each output sample is aligned with each input sample
    and *Y* is the same size as *X*.  If len(h) is even, each output sample is
    aligned with the mid point of each pair of input samples, and Y.shape =
    X.shape + [1 0].
    :param X: an image whose columns are to be filtered
    :param h: the filter coefficients.
    :returns Y: the filtered image.
    .. codeauthor:: Rich Wareham <rjw57@cantab.net>, August 2013
    .. codeauthor:: Cian Shaffrey, Cambridge University, August 2000
    .. codeauthor:: Nick Kingsbury, Cambridge University, August 2000
    """

    m = h.get_shape().as_list()[0]
    m2 = m // 2

    # Symmetrically extend with repeat of end samples.
    # Pad only the second dimension of the tensor X (the columns)
    X = tf.pad(X, [[0, 0], [m2, m2], [0, 0]], 'SYMMETRIC')
    r, c = X.get_shape().as_list()[1:3]

    # X currently has shape [batch, rows, cols]
    # h currently has shape [f_rows]
    # For conv2d to work, X needs to be in shape [batch, rows, cols, in_channels]
    # and h needs to be in shape [f_rows, f_cols, in_channels, out_channels]
    h = tf.reshape(h, [-1, 1, 1, 1])
    X = tf.expand_dims(X, axis=-1)

    y = tf.nn.conv2d(X, h, strides=[1, 1, 1, 1], padding='VALID')
    r, c = y.get_shape().as_list()[1:3]
    # Drop the last dimension
    return tf.reshape(y, [-1, r, c])


def rowfilter(X, h):
    """Filter the rows of image *X* using filter vector *h*, without decimation.
    If len(h) is odd, each output sample is aligned with each input sample
    and *Y* is the same size as *X*.  If len(h) is even, each output sample is
    aligned with the mid point of each pair of input samples, and Y.shape =
    X.shape + [0 1].
    :param X: an image whose columns are to be filtered
    :param h: the filter coefficients.
    :returns Y: the filtered image.
    .. codeauthor:: Rich Wareham <rjw57@cantab.net>, August 2013
    .. codeauthor:: Cian Shaffrey, Cambridge University, August 2000
    .. codeauthor:: Nick Kingsbury, Cambridge University, August 2000
    """

    m = h.get_shape().as_list()[0]
    m2 = m // 2

    # Symmetrically extend with repeat of end samples.
    # Pad only the second dimension of the tensor X (the columns)
    X = tf.pad(X, [[0, 0], [0, 0], [m2, m2]], 'SYMMETRIC')
    r, c = X.get_shape().as_list()[1:3]

    # X currently has shape [batch, rows, cols]
    # h currently has shape [f_rows]
    # For conv2d to work, X needs to be in shape [batch, rows, cols, in_channels]
    # and h needs to be in shape [f_rows, f_cols, in_channels, out_channels]
    h = tf.reshape(h, [1, -1, 1, 1])
    X = tf.expand_dims(X, axis=-1)

    y = tf.nn.conv2d(X, h, strides=[1, 1, 1, 1], padding='VALID')
    r, c = y.get_shape().as_list()[1:3]
    # Drop the last dimension
    return tf.reshape(y, [-1, r, c])

In [ ]:
def coldfilt(X, ha, hb, a_first=True):
    """Filter the columns of image X using the two filters ha and hb =
    reverse(ha). 
    ha operates on the odd samples of X and hb on the even samples.  
    Both filters should be even length, and h should be approx linear
    phase with a quarter sample (i.e. an :math:`e^{j \pi/4}`) advance from 
    its mid pt (i.e. :math:`|h(m/2)| > |h(m/2 + 1)|`).
    .. code-block:: text
                          ext        top edge                     bottom edge       ext
        Level 1:        !               |               !               |               !
        odd filt on .    b   b   b   b   a   a   a   a   a   a   a   a   b   b   b   b
        odd filt on .      a   a   a   a   b   b   b   b   b   b   b   b   a   a   a   a
        Level 2:        !               |               !               |               !
        +q filt on x      b       b       a       a       a       a       b       b
        -q filt on o          a       a       b       b       b       b       a       a
    The output is decimated by two from the input sample rate and the results
    from the two filters, Ya and Yb, are interleaved to give Y.  
    Symmetric extension with repeated end samples is used on the composite X columns
    before each filter is applied.
    Raises ValueError if the number of rows in X is not a multiple of 4, the
    length of ha does not match hb or the lengths of ha or hb are non-even.
    .. codeauthor:: Rich Wareham <rjw57@cantab.net>, August 2013
    .. codeauthor:: Cian Shaffrey, Cambridge University, August 2000
    .. codeauthor:: Nick Kingsbury, Cambridge University, August 2000
    """

    r, c = X.get_shape().as_list()[1:]
    r2 = r // 2
    if r % 4 != 0:
        raise ValueError('No. of rows in X must be a multiple of 4')

    if ha.shape != hb.shape:
        raise ValueError('Shapes of ha and hb must be the same')

    if ha.get_shape().as_list()[0] % 2 != 0:
        raise ValueError('Lengths of ha and hb must be even')

    m = ha.get_shape().as_list()[0]
    m2 = m // 2

    # Symmetrically extend with repeat of end samples.
    # Pad only the second dimension of the tensor X (the columns).
    # SYMMETRIC extension means the edge sample is repeated twice, whereas
    # REFLECT only has the edge sample once    
    X = tf.pad(X, [[0, 0], [m, m], [0, 0]], 'SYMMETRIC')    
    '''
    # Perform filtering on columns of extended matrix X  current shape: [Batch, r+2*m, c]
    # We split X into 4 polyphase representations, and apply ha to the odd phases and apply hb to the even phases.
    # These will each be of size [Batch, r/4 + m/2 - 1, c, 1]    
    phase1 = tf.expand_dims(X[:,2:r+2*m-2:4,:], axis=-1)
    phase2 = tf.expand_dims(X[:,3:r+2*m-2:4,:], axis=-1)
    phase3 = tf.expand_dims(X[:,4:r+2*m-2:4,:], axis=-1)
    phase4 = tf.expand_dims(X[:,5:r+2*m-2:4,:], axis=-1)
    
    # To massage them into the shape needed for conv2d, we pack:
    # the odd  phases into X_odd of size  [Batch, r/4+m/2-1, c, 2] and 
    # the even phases into X_even of size [Batch, r/4+m/2-1, c, 2] and
    # then apply convolution, using 'valid' padding
    
    # Select odd and even samples from ha and hb. Note that due to 0-indexing
    # 'odd' and 'even' are not perhaps what you might expect them to be.
    hao = tf.reshape(ha[0:m:2], [-1, 1, 1, 1])
    hae = tf.reshape(ha[1:m:2], [-1, 1, 1, 1])
    hbo = tf.reshape(hb[0:m:2], [-1, 1, 1, 1])
    hbe = tf.reshape(hb[1:m:2], [-1, 1, 1, 1])  
    print(hao.shape)
    print(X.shape)
    a_rows = tf.nn.conv2d(phase3, hae, strides=[1,1,1,1], padding='VALID') + \
             tf.nn.conv2d(phase1, hao, strides=[1,1,1,1], padding='VALID')
    b_rows = tf.nn.conv2d(phase2, hae, strides=[1,1,1,1], padding='VALID') + \
             tf.nn.conv2d(phase4, hao, strides=[1,1,1,1], padding='VALID')
    return a_rows, b_rows
    """
    '''
    X_odd = tf.expand_dims(X[:,2:r+2*m-2:2,:], axis=-1)
    X_even =tf.expand_dims(X[:,3:r+2*m-2:2,:], axis=-1)
    ha = tf.reshape(ha, [m,1,1,1])
    hb = tf.reshape(hb, [m,1,1,1])
    a_rows = tf.nn.conv2d(X_odd, ha, strides=[1,2,1,1], padding='VALID')
    b_rows = tf.nn.conv2d(X_even, hb, strides=[1,2,1,1], padding='VALID')
    
    # We interleave the two results into a tensor of size [Batch, r/2, c]
    # Concat a_rows and b_rows (both of shape [Batch, r/4, c, 1]) 
    Y = tf.cond(tf.reduce_sum(ha*hb) > 0,
            lambda: tf.concat([a_rows,b_rows],axis=-1),
            lambda: tf.concat([b_rows,a_rows],axis=-1))
    '''
    if a_first:
        Y = tf.concat([a_rows,b_rows],axis=-1)
    else:
        Y = tf.concat([b_rows,a_rows],axis=-1)
    '''
    
    # Permute result to be shape [Batch, r/4, 2, c]
    Y = tf.transpose(Y, perm=[0,1,3,2])
    
    # Reshape result to be shape [Batch, r/2, c]. This reshaping interleaves
    # the columns
    Y = tf.reshape(Y, [-1, r2, c])   
    
    return Y



In [ ]:
def rowdfilt(X, ha, hb, a_first=True):
    """Filter the rows of image X using the two filters ha and hb =
    reverse(ha).  ha operates on the odd samples of X and hb on the even
    samples.  Both filters should be even length, and h should be approx linear
    phase with a quarter sample advance from its mid pt (i.e. :math:`|h(m/2)| >
    |h(m/2 + 1)|`).
    .. code-block:: text
                          ext        top edge                     bottom edge       ext
        Level 1:        !               |               !               |               !
        odd filt on .    b   b   b   b   a   a   a   a   a   a   a   a   b   b   b   b
        odd filt on .      a   a   a   a   b   b   b   b   b   b   b   b   a   a   a   a
        Level 2:        !               |               !               |               !
        +q filt on x      b       b       a       a       a       a       b       b
        -q filt on o          a       a       b       b       b       b       a       a
    The output is decimated by two from the input sample rate and the results
    from the two filters, Ya and Yb, are interleaved to give Y.  Symmetric
    extension with repeated end samples is used on the composite X rows
    before each filter is applied.
    Raises ValueError if the number of columns in X is not a multiple of 4, the
    length of ha does not match hb or the lengths of ha or hb are non-even.
    .. codeauthor:: Rich Wareham <rjw57@cantab.net>, August 2013
    .. codeauthor:: Cian Shaffrey, Cambridge University, August 2000
    .. codeauthor:: Nick Kingsbury, Cambridge University, August 2000
    """

    r, c = X.get_shape().as_list()[1:]
    c2 = c // 2
    if c % 4 != 0:
        raise ValueError('No. of rows in X must be a multiple of 4')

    if ha.shape != hb.shape:
        raise ValueError('Shapes of ha and hb must be the same')

    if ha.get_shape().as_list()[0] % 2 != 0:
        raise ValueError('Lengths of ha and hb must be even')

    m = ha.get_shape().as_list()[0]

    # Symmetrically extend with repeat of end samples.
    # Pad only the second dimension of the tensor X (the rows).
    # SYMMETRIC extension means the edge sample is repeated twice, whereas
    # REFLECT only has the edge sample once
    
    X = tf.pad(X, [[0, 0], [0, 0], [m, m]], 'SYMMETRIC')
    '''
    # Perform filtering on columns of extended matrix X  current shape: [Batch, r, c+2*m]
    # We split X into 4 polyphase representations, and apply ha to the odd phases and apply hb to the even phases.
    # These will each be of size [Batch, r, c/4 + m/2 - 1, 1]
    t = np.arange(5, c+2*m-2, 4, dtype=np.int32)
    phase1 = tf.expand_dims(X[:,:,t-3], axis=-1)
    phase2 = tf.expand_dims(X[:,:,t-2], axis=-1)
    phase3 = tf.expand_dims(X[:,:,t-1], axis=-1)
    phase4 = tf.expand_dims(X[:,:,t], axis=-1)
    
    # To massage them into the shape needed for conv2d, we pack:
    # the odd  phases into X_odd of size  [Batch, r, c/4 + m/2 -1, 2] and 
    # the even phases into X_even of size [Batch, r, c/4 + m/2 -1, 2] and
    # then apply convolution, using 'valid' padding
    
    # Select odd and even samples from ha and hb. Note that due to 0-indexing
    # 'odd' and 'even' are not perhaps what you might expect them to be.
    hao = tf.reshape(ha[0:m:2], [1, -1, 1, 1])
    hae = tf.reshape(ha[1:m:2], [1, -1, 1, 1])
    hbo = tf.reshape(hb[0:m:2], [1, -1, 1, 1])
    hbe = tf.reshape(hb[1:m:2], [1, -1, 1, 1])  
    a_cols = tf.nn.conv2d(phase1, hae, strides=[1,1,1,1], padding='VALID') + \
             tf.nn.conv2d(phase3, hao, strides=[1,1,1,1], padding='VALID')
    b_cols = tf.nn.conv2d(phase2, hae, strides=[1,1,1,1], padding='VALID') + \
             tf.nn.conv2d(phase4, hao, strides=[1,1,1,1], padding='VALID')
        
    """
    # Could also try:
    
    odd_phases = tf.concat([phase3, phase1], axis=-1)
    even_phases = tf.concat([phase4, phase2], axis=-1)
    ha_split = tf.reshape(ha, [1,m2,2,1])
    hb_split = tf.reshape(hb, [1,m2,2,1])
    a_cols = tf.nn.conv2d(odd_phases, ha_split, strides=[1,1,1,1], padding='VALID')
    b_cols = tf.nn.conv2d(even_phases, hb_split, strides=[1,1,1,1], padding='VALID')    
    """
    '''
    X_odd = tf.expand_dims(X[:,:,2:c+2*m-2:2], axis=-1)
    X_even =tf.expand_dims(X[:,:,3:c+2*m-2:2], axis=-1)
    ha = tf.reshape(ha, [1,m,1,1])
    hb = tf.reshape(hb, [1,m,1,1])
    a_cols = tf.nn.conv2d(X_odd, ha, strides=[1,1,2,1], padding='VALID')
    b_cols = tf.nn.conv2d(X_even, hb, strides=[1,1,2,1], padding='VALID')
    
    # We interleave the two results into a tensor of size [Batch, r/2, c]
    # Concat a_cols and b_cols (both of shape [Batch, r, c/4, 1])      
    Y = tf.cond(tf.reduce_sum(ha*hb) > 0,
                lambda: tf.concat([a_cols,b_cols],axis=-1),
                lambda: tf.concat([b_cols,a_cols],axis=-1))
    '''
    if a_first:
        Y = tf.concat([a_cols,b_cols],axis=-1)
    else:
        Y = tf.concat([b_cols,a_cols],axis=-1)
    ''' 
    # Reshape result to be shape [Batch, r, c/2]. This reshaping interleaves
    # the columns
    Y = tf.reshape(Y, [-1, r, c2])   
    
    return Y